In [2]:
!pip install torchvision

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/4c/6a/c7752603060d076dfed95135b78b047dc71792630cbcb022e3693d6f32ef/torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl (1.6 MB)



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\Acer\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [1]:
!pip install scenedetect[opencv]

  Obtaining dependency information for scenedetect[opencv] from https://files.pythonhosted.org/packages/b1/5b/c090fe55521265eb1816c303267e985125000a4e32237e95562ed462608f/scenedetect-0.6.6-py3-none-any.whl.metadata
  Obtaining dependency information for Click from https://files.pythonhosted.org/packages/7e/d4/7ebdbd03970677812aac39c869717059dbb71a4cfc033ca6e5221787892c/click-8.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a4/7d/f1c30a92854540bf789e9cd5dde7ef49bbe63f855b85a2e6b3db8135c591/opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ------------ --------------------------- 30.7/98.2 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/39.5 MB 2.4 MB/s eta 0


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\Acer\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [17]:
import os
import shutil
import numpy as np
import cv2
from PIL import Image

# PyTorch and torchvision imports
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

# Scenedetect and config imports remain unchanged
from scenedetect import VideoManager, SceneManager, StatsManager
from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import save_images, write_scene_list_html
import config
import json
import random

In [18]:
# # ------------------------------
# # Set up directories based on current working directory
# # ------------------------------
# base_dir = os.path.join(os.getcwd(), "features")  # base directory for your data
# video_dir = os.path.join(base_dir, "videos_train")        # Directory containing input videos
# feat_dir = os.path.join(base_dir, "feat_train")        # Directory to save feature .npy files
# temp_dir = os.path.join(base_dir, "temporary_images") # Temporary folder for frame extraction
# scenes_dir = os.path.join(base_dir, "scenes")         # Directory to save scene images

# # Create necessary directories if they don't exist
# os.makedirs(feat_dir, exist_ok=True)
# os.makedirs(scenes_dir, exist_ok=True)

# # ------------------------------
# # Set device (CPU if no CUDA available)
# # ------------------------------
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

# # ------------------------------
# # Define the PyTorch Model Wrapper for EfficientNetB7
# # ------------------------------
# class EfficientNetB7_Features(nn.Module):
#     """
#     Wraps torchvision's EfficientNetB7 to output features from the penultimate layer.
#     """
#     def __init__(self, base_model):
#         super(EfficientNetB7_Features, self).__init__()
#         self.features = base_model.features
#         self.avgpool = base_model.avgpool
#         self.dropout = base_model.classifier[0]  # keep the dropout output
        
#     def forward(self, x):
#         x = self.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.dropout(x)
#         return x

# def model_cnn_load():
#     """
#     Loads a pretrained EfficientNetB7 model from torchvision,
#     wraps it to output feature vectors, and moves it to the appropriate device.
#     """
#     base_model = models.efficientnet_b7(pretrained=True)
#     model_final = EfficientNetB7_Features(base_model)
#     model_final.to(device)
#     print("Model loaded:")
#     print(model_final)
#     return model_final

# # ------------------------------
# # Define image transformation pipeline
# # ------------------------------
# transform = transforms.Compose([
#     transforms.Resize((600, 600)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# def load_image(path):
#     """
#     Loads an image using PIL and applies the transformation.
#     """
#     img = Image.open(path).convert('RGB')
#     return transform(img)

# # ------------------------------
# # Frame extraction from video segments
# # ------------------------------
# def video_to_frames(video, startT, stopT, video_directory, tmp_dir):
#     """
#     Extracts frames from a video between startT and stopT (in seconds).
    
#     Parameters:
#       video         : Video filename (e.g., 'sample.avi')
#       startT, stopT : Start and stop times (in seconds) for extraction.
#       video_directory: Directory where video files are stored.
#       tmp_dir       : Temporary directory to store extracted frames.
      
#     Returns:
#       List of file paths to the extracted frames.
#     """
#     startT_ms = startT * 1000
#     stopT_ms = stopT * 1000

#     if os.path.exists(tmp_dir):
#         shutil.rmtree(tmp_dir)
#     os.makedirs(tmp_dir)
    
#     video_path = os.path.join(video_directory, video)
#     cap = cv2.VideoCapture(video_path)
#     count = 0
#     image_list = []
#     success = True
    
#     # Skip frames until reaching startT
#     while success and cap.get(cv2.CAP_PROP_POS_MSEC) < startT_ms:
#         success, _ = cap.read()
    
#     # Extract frames until stopT is reached
#     while success and cap.get(cv2.CAP_PROP_POS_MSEC) <= stopT_ms:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame_path = os.path.join(tmp_dir, f'frame{count}.jpg')
#         cv2.imwrite(frame_path, frame)
#         image_list.append(frame_path)
#         count += 1
        
#     cap.release()
#     print(f"Extracted {len(image_list)} frames from {video}")
#     return image_list

# # ------------------------------
# # Feature extraction for a video scene
# # ------------------------------
# def extract_features(video, model, startT, stopT, video_directory, tmp_dir):
#     """
#     Extracts features for a video scene defined by startT and stopT.
#     Uniformly samples 20 frames, processes them, and passes them through the CNN model.
    
#     Returns a NumPy array containing the extracted features.
#     """
#     image_list = video_to_frames(video, startT, stopT, video_directory, tmp_dir)
#     if len(image_list) == 0:
#         print("No frames extracted!")
#         return None
#     samples = np.round(np.linspace(0, len(image_list) - 1, 20)).astype(int)
#     sampled_images = [image_list[i] for i in samples]
    
#     images = [load_image(p) for p in sampled_images]
#     images = torch.stack(images).to(device)
    
#     model.eval()
#     with torch.no_grad():
#         features = model(images)
#     features = features.cpu().numpy()
    
#     # Clean up temporary directory
#     shutil.rmtree(tmp_dir)
#     return features

# # ------------------------------
# # Scene detection using scenedetect
# # ------------------------------
# def proposal_module(video, video_directory):
#     """
#     Detects scenes in a video using scenedetect.
#     Saves representative scene images in the 'scenes' directory and an HTML summary.
    
#     Returns:
#       A list of timestamps (in seconds). For each scene, two values are appended: start and end.
#     """
#     video_path = os.path.join(video_directory, video)
#     stats_path = os.path.join(os.getcwd(), "result.csv")
    
#     video_manager = VideoManager([video_path])
#     stats_manager = StatsManager()
#     scene_manager = SceneManager(stats_manager)
    
#     scene_manager.add_detector(ContentDetector(threshold=30))
#     video_manager.set_downscale_factor()
    
#     video_manager.start()
#     scene_manager.detect_scenes(frame_source=video_manager)
    
#     with open(stats_path, 'w') as f:
#         stats_manager.save_to_csv(f, video_manager.get_base_timecode())
    
#     scene_list = scene_manager.get_scene_list()
#     print(f"Detected {len(scene_list)} scenes in {video}")
    
#     # Save scene images and HTML summary
#     save_images(scene_list, video_manager, num_images=1, image_name_template='$SCENE_NUMBER', output_dir=scenes_dir)
#     if scene_list:
#         try:
#             write_scene_list_html(os.path.join(os.getcwd(), "result.html"), scene_list)
#         except Exception as e:
#             print("Warning: Could not write HTML summary:", e)
    
#     scenes = []
#     for scene in scene_list:
#         start, end = scene
#         scenes.append(float(start.get_seconds()))
#         scenes.append(float(end.get_seconds()))
    
#     return scenes

# # ------------------------------
# # Main feature extraction routine
# # ------------------------------
# def extract_feats_pretrained_cnn():
#     """
#     Iterates over all videos in the video directory, detects scenes,
#     extracts features for each scene, and saves them as .npy files.
#     Also writes out a JSON mapping file of video IDs to feature file names.
#     """
#     model = model_cnn_load()
#     video_list = os.listdir(video_dir)
    
#     # For testing purposes, you can limit the number of videos processed.
#     video_list = video_list[:4]
    
#     feature_mapping = {}  # to map video IDs to feature file names
    
#     for video in video_list:
#         video_id = os.path.splitext(video)[0]
#         print(f"Processing video: {video}")
        
#         # Detect scenes using scenedetect
#         scenes = proposal_module(video, video_dir)
#         if not scenes or len(scenes) < 2:
#             print(f"Not enough scene boundaries detected for {video}. Skipping...")
#             continue
        
#         feature_mapping[video_id] = []
#         count = 0
#         # Process each detected scene (start and end timestamps)
#         for i in range(0, len(scenes), 2):
#             count += 1
#             if count == 1:
#                 outfile = os.path.join(feat_dir, f"{video_id}.npy")
#             else:
#                 outfile = os.path.join(feat_dir, f"{video_id}_{count}.npy")
#             print(f"Extracting features for scene {count} of {video}")
            
#             feats = extract_features(video, model, scenes[i], scenes[i+1], video_dir, temp_dir)
#             if feats is not None:
#                 np.save(outfile, feats)
#                 print(f"Saved features to {outfile}")
#                 feature_mapping[video_id].append(outfile)
#             else:
#                 print(f"Feature extraction failed for scene {count} of {video}")
    
#     # Write the feature mapping to a JSON file
#     mapping_file = os.path.join(feat_dir, "feat_mapping.json")
#     with open(mapping_file, "w") as f:
#         json.dump(feature_mapping, f, indent=2)
#     print(f"Feature mapping saved to {mapping_file}")

# if __name__ == "__main__":
#     extract_feats_pretrained_cnn()


In [19]:

# ------------------------------
# Set up directories based on current working directory
# ------------------------------
base_dir = os.path.join(os.getcwd(), "features")  # Base directory for all features and videos

# Original videos are assumed to be stored in "videos_all"
videos_all_dir = os.path.join(base_dir, "videos")
# Create separate directories for training and test videos
videos_train_dir = os.path.join(base_dir, "videos_train")
videos_test_dir  = os.path.join(base_dir, "videos_test")

# Directories for saving extracted features
feat_train_dir = os.path.join(base_dir, "feat_train")
feat_test_dir  = os.path.join(base_dir, "feat_test")

# Other directories
temp_dir = os.path.join(base_dir, "temporary_images")  # Temporary folder for frame extraction
scenes_dir = os.path.join(base_dir, "scenes")            # Directory to save scene images

# Create necessary directories if they don't exist
os.makedirs(videos_all_dir, exist_ok=True)
os.makedirs(videos_train_dir, exist_ok=True)
os.makedirs(videos_test_dir, exist_ok=True)
os.makedirs(feat_train_dir, exist_ok=True)
os.makedirs(feat_test_dir, exist_ok=True)
os.makedirs(scenes_dir, exist_ok=True)
# temp_dir will be created/deleted on the fly

# ------------------------------
# Set device (CPU if no CUDA available)
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ------------------------------
# Define the PyTorch Model Wrapper for EfficientNetB7
# ------------------------------
class EfficientNetB7_Features(nn.Module):
    """
    Wraps torchvision's EfficientNetB7 to output features from the penultimate layer.
    """
    def __init__(self, base_model):
        super(EfficientNetB7_Features, self).__init__()
        self.features = base_model.features
        self.avgpool = base_model.avgpool
        self.dropout = base_model.classifier[0]  # keep the dropout output
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return x

def model_cnn_load():
    """
    Loads a pretrained EfficientNetB7 model from torchvision,
    wraps it to output feature vectors, and moves it to the appropriate device.
    """
    base_model = models.efficientnet_b7(pretrained=True)
    model_final = EfficientNetB7_Features(base_model)
    model_final.to(device)
    print("Model loaded:")
    print(model_final)
    return model_final

# ------------------------------
# Define image transformation pipeline
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def load_image(path):
    """
    Loads an image using PIL and applies the transformation.
    """
    img = Image.open(path).convert('RGB')
    return transform(img)

# ------------------------------
# Frame extraction from video segments
# ------------------------------
def video_to_frames(video, startT, stopT, video_directory, tmp_dir):
    """
    Extracts frames from a video between startT and stopT (in seconds).
    Returns a list of file paths to the extracted frames.
    """
    startT_ms = startT * 1000
    stopT_ms = stopT * 1000

    if os.path.exists(tmp_dir):
        shutil.rmtree(tmp_dir)
    os.makedirs(tmp_dir)
    
    video_path = os.path.join(video_directory, video)
    cap = cv2.VideoCapture(video_path)
    count = 0
    image_list = []
    success = True
    
    # Skip frames until reaching startT
    while success and cap.get(cv2.CAP_PROP_POS_MSEC) < startT_ms:
        success, _ = cap.read()
    
    # Extract frames until stopT is reached
    while success and cap.get(cv2.CAP_PROP_POS_MSEC) <= stopT_ms:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(tmp_dir, f'frame{count}.jpg')
        cv2.imwrite(frame_path, frame)
        image_list.append(frame_path)
        count += 1
        
    cap.release()
    print(f"Extracted {len(image_list)} frames from {video}")
    return image_list

# ------------------------------
# Feature extraction for a video scene
# ------------------------------
def extract_features(video, model, startT, stopT, video_directory, tmp_dir):
    """
    Extracts features for a video scene defined by startT and stopT.
    Uniformly samples 20 frames, processes them, and passes them through the CNN model.
    Returns a NumPy array containing the extracted features.
    """
    image_list = video_to_frames(video, startT, stopT, video_directory, tmp_dir)
    if len(image_list) == 0:
        print("No frames extracted!")
        return None
    samples = np.round(np.linspace(0, len(image_list) - 1, 20)).astype(int)
    sampled_images = [image_list[i] for i in samples]
    
    images = [load_image(p) for p in sampled_images]
    images = torch.stack(images).to(device)
    
    model.eval()
    with torch.no_grad():
        features = model(images)
    features = features.cpu().numpy()
    
    # Clean up temporary directory
    shutil.rmtree(tmp_dir)
    return features

# ------------------------------
# Scene detection using scenedetect
# ------------------------------
def proposal_module(video, video_directory):
    """
    Detects scenes in a video using scenedetect.
    Saves representative scene images in the 'scenes' directory and an HTML summary.
    Returns a list of timestamps (in seconds); for each scene, two values (start and end) are appended.
    """
    # Assume VideoManager, StatsManager, SceneManager, ContentDetector, save_images, write_scene_list_html are imported
    video_path = os.path.join(video_directory, video)
    stats_path = os.path.join(os.getcwd(), "result.csv")
    
    video_manager = VideoManager([video_path])
    stats_manager = StatsManager()
    scene_manager = SceneManager(stats_manager)
    
    scene_manager.add_detector(ContentDetector(threshold=30))
    video_manager.set_downscale_factor()
    
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)
    
    with open(stats_path, 'w') as f:
        stats_manager.save_to_csv(f, video_manager.get_base_timecode())
    
    scene_list = scene_manager.get_scene_list()
    print(f"Detected {len(scene_list)} scenes in {video}")
    
    # Save scene images and HTML summary
    save_images(scene_list, video_manager, num_images=1, image_name_template='$SCENE_NUMBER', output_dir=scenes_dir)
    if scene_list:
        try:
            write_scene_list_html(os.path.join(os.getcwd(), "result.html"), scene_list)
        except Exception as e:
            print("Warning: Could not write HTML summary:", e)
    
    scenes = []
    for scene in scene_list:
        start, end = scene
        scenes.append(float(start.get_seconds()))
        scenes.append(float(end.get_seconds()))
    
    return scenes

# ------------------------------
# Main routine: Train-Test Split and Feature Extraction
# ------------------------------
def extract_feats_with_split():
    """
    Splits videos from the 'videos_all' folder into training and testing sets,
    copies them to respective folders, extracts features for each video scene,
    and writes out mapping files and text files listing the videos.
    """
    # Define the train-test split ratio
    train_ratio = 0.85

    # Get list of all videos from videos_all_dir
    all_videos = [f for f in os.listdir(videos_all_dir) if os.path.isfile(os.path.join(videos_all_dir, f))]
    random.shuffle(all_videos)
    split_index = int(len(all_videos) * train_ratio)
    train_videos = all_videos[:split_index]
    test_videos = all_videos[split_index:]
    
    # Copy videos to separate directories
    for video in train_videos:
        shutil.copy(os.path.join(videos_all_dir, video), os.path.join(videos_train_dir, video))
    for video in test_videos:
        shutil.copy(os.path.join(videos_all_dir, video), os.path.join(videos_test_dir, video))
    
    # Write out lists of video IDs
    with open(os.path.join(base_dir, "train_videos.txt"), "w") as f:
        for video in train_videos:
            f.write(os.path.splitext(video)[0] + "\n")
    with open(os.path.join(base_dir, "test_videos.txt"), "w") as f:
        for video in test_videos:
            f.write(os.path.splitext(video)[0] + "\n")
    
    # Initialize model
    model = model_cnn_load()
    
    # Process training videos
    train_mapping = {}
    print("Extracting features for training videos...")
    for video in train_videos:
        video_id = os.path.splitext(video)[0]
        print(f"Processing training video: {video}")
        scenes = proposal_module(video, videos_train_dir)
        if not scenes or len(scenes) < 2:
            print(f"Not enough scene boundaries detected for {video}. Skipping...")
            continue
        train_mapping[video_id] = []
        count = 0
        for i in range(0, len(scenes), 2):
            count += 1
            if count == 1:
                outfile = os.path.join(feat_train_dir, f"{video_id}.npy")
            else:
                outfile = os.path.join(feat_train_dir, f"{video_id}_{count}.npy")
            print(f"Extracting features for scene {count} of {video}")
            feats = extract_features(video, model, scenes[i], scenes[i+1], videos_train_dir, temp_dir)
            if feats is not None:
                np.save(outfile, feats)
                print(f"Saved features to {outfile}")
                train_mapping[video_id].append(outfile)
            else:
                print(f"Feature extraction failed for scene {count} of {video}")
    
    # Save train mapping JSON
    train_mapping_file = os.path.join(feat_train_dir, "feat_mapping_train.json")
    with open(train_mapping_file, "w") as f:
        json.dump(train_mapping, f, indent=2)
    print(f"Training feature mapping saved to {train_mapping_file}")
    
    # Process test videos
    test_mapping = {}
    print("Extracting features for test videos...")
    for video in test_videos:
        video_id = os.path.splitext(video)[0]
        print(f"Processing test video: {video}")
        scenes = proposal_module(video, videos_test_dir)
        if not scenes or len(scenes) < 2:
            print(f"Not enough scene boundaries detected for {video}. Skipping...")
            continue
        test_mapping[video_id] = []
        count = 0
        for i in range(0, len(scenes), 2):
            count += 1
            if count == 1:
                outfile = os.path.join(feat_test_dir, f"{video_id}.npy")
            else:
                outfile = os.path.join(feat_test_dir, f"{video_id}_{count}.npy")
            print(f"Extracting features for scene {count} of {video}")
            feats = extract_features(video, model, scenes[i], scenes[i+1], videos_test_dir, temp_dir)
            if feats is not None:
                np.save(outfile, feats)
                print(f"Saved features to {outfile}")
                test_mapping[video_id].append(outfile)
            else:
                print(f"Feature extraction failed for scene {count} of {video}")
    
    # Save test mapping JSON
    test_mapping_file = os.path.join(feat_test_dir, "feat_mapping_test.json")
    with open(test_mapping_file, "w") as f:
        json.dump(test_mapping, f, indent=2)
    print(f"Test feature mapping saved to {test_mapping_file}")

if __name__ == "__main__":
    extract_feats_with_split()


Using device: cpu


KeyboardInterrupt: 